## README
This notebook is for processing the data generated by the IMR Batteries scrapper. Before using this notebook, run the scraper script to generate `18650_data.json` file.

In [3]:
import pandas as pd
import scipy.optimize as spo

In [4]:
df = pd.read_json('18650_data.json')

In [5]:
# df['Energy Density (mAh/g)'] = df['Nominal Capacity (mAh)'] / df['Weight (g)']
df['Energy (Wh)'] = df['Nominal Voltage'] * (df['Nominal Capacity (mAh)'] / 1000)
df['Energy Density (Wh/g)'] = df['Energy (Wh)'] / df['Weight (g)']

In [6]:
df.sort_values('Energy Density (Wh/g)', ascending=False)

,Brand,Dimensions,Discharge Current (A),Maximum Voltage,Model,Nominal Capacity (mAh),Nominal Voltage,Protected,Rated Capacity,Rechargeable,Size,Source,Style,Weight (g),Energy (Wh),Energy Density (Wh/g)
14,IMREN,18.39mm x 65.27mm,10.00,4.2,NaN,3500,3.70,"NO, UNPROTECTED",NaN,Yes,18650,https://www.imrbatteries.com/imren-18650-3500m...,Flat,46.90,12.9500,0.276119
7,Efest,18.25mm x 65.25mm,10.00,4.2,,3500,3.70,"NO, UNPROTECTED",NaN,Yes,18650,https://www.imrbatteries.com/efest-18650-3500m...,Flat Top,47.00,12.9500,0.275532
39,MXJO,18.30mm x 65.36mm,10.00,4.2,NaN,3500,3.70,"NO, UNPROTECTED",NaN,NaN,18650,https://www.imrbatteries.com/mxjo-18650-3500ma...,Flat Top,47.10,12.9500,0.274947
1,AWT,18.4mm x 65.3mm,10.00,4.2,NaN,3500,3.70,"NO, UNPROTECTED",NaN,Yes,18650,https://www.imrbatteries.com/awt-18650-3500mah...,Flat,47.90,12.9500,0.270355
68,Samsung SDI,18.3mm x 65.3mm,10.00,4.2,36G,3600,3.60,"NO, UNPROTECTED",,NaN,18650,https://www.imrbatteries.com/samsung-36g-18650...,Flat Top,48.00,12.9600,0.270000
31,LG Chem,18.30mm x 65.10mm,10.00,4.2,MJ1,3500,3.60,"NO, UNPROTECTED",NaN,NaN,18650,https://www.imrbatteries.com/lg-mj1-18650-3500...,Button/Nipple Top,46.90,12.6000,0.268657
15,LG Chem,18.4mm x 65mm,4.87,4.2,F1L,3350,3.60,"NO, UNPROTECTED",NaN,NaN,18650,https://www.imrbatteries.com/lg-f1l-18650-3350...,Flat Top,45.00,12.0600,0.268000
76,Sony Energy,18.1mm x 64.9mm,8.00,4.2,US18650VC7,3500,3.60,"NO, UNPROTECTED",NaN,NaN,18650,https://www.imrbatteries.com/sony-vc7-18650-35...,Flat,47.10,12.6000,0.267516
4,EFAN,18.5mm x 65.1mm,10.00,4.2,NaN,3500,3.70,"NO, UNPROTECTED",NaN,Yes,18650,https://www.imrbatteries.com/efan-18650-3500ma...,Flat Top,48.50,12.9500,0.267010
72,Sanyo,18.31mm x 65.12mm,7.00,4.2,UR18650BL,3350,3.60,"NO, UNPROTECTED",3200mAh,NaN,18650,https://www.imrbatteries.com/sanyo-ur18650bl-3...,Flat Top,45.30,12.0600,0.266225


In [7]:
df[df['Model'] == '36G']

,Brand,Dimensions,Discharge Current (A),Maximum Voltage,Model,Nominal Capacity (mAh),Nominal Voltage,Protected,Rated Capacity,Rechargeable,Size,Source,Style,Weight (g),Energy (Wh),Energy Density (Wh/g)
68,Samsung SDI,18.3mm x 65.3mm,10.0,4.2,36G,3600,3.6,"NO, UNPROTECTED",,NaN,18650,https://www.imrbatteries.com/samsung-36g-18650...,Flat Top,48.0,12.96,0.27


In [14]:
def compute_pack_data(config, cell):
    """"""
    
    s = config[0]
    p = config[1]
    data = {}
    data['Maximum Voltage'] = cell['Maximum Voltage'] * s
    data['Nominal Voltage'] = cell['Nominal Voltage'] * s
    data['Weight (g)'] = cell['Weight (g)'] * s * p
    data['Energy (Wh)'] = cell['Energy (Wh)'] * s * p
    data['Nominal Capacity (mAh)'] = cell['Nominal Capacity (mAh)'] * p
    data['Discharge Current (A)'] = cell['Discharge Current (A)'] * p
    # return pd.DataFrame(data)
    return data['Weight (g)']
compute_pack_data((130, 30), df[df['Model'] == '36G'])

68    187200.0
Name: Weight (g), dtype: float64

In [13]:
# solver doesn't work
spo.minimize(compute_pack_data,[130,30],(df[df['Model'] == '36G'],))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().